# **PNAS2020**


This script is an example file showing how to import atmospheric river data used in the paper : ***Seasonal Prediction of western North America Atmospheric Rivers from Github***. One should notice that due to the storage limitation of Github and policy of GFDL, all files are saved with a post-process form (i.e., AR). If you are interested in the raw data for detecting ARs (e.g., u, v and q), please contact corresponding author **Kai-Chih Tseng**: kaichiht@princeton.edu


Code instruction:

1.   Please copy this file to your googledrive (or local machine) before using it
2.  ram for analyzing this data can't be less than 25GB



## Step 1: Setup environment

In [1]:
# environment setup
!pip install netcdf4
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from netCDF4 import Dataset as NetCDFFile
import random 
import timeit
import pickle
from scipy import stats
!pip install wget
import wget

     |████████████████████████████████| 4.3MB 2.7MB/s 
     |████████████████████████████████| 296kB 51.2MB/s 
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=2646278c26d10ce1c6fd5697c71b73eac754e90ee2a47682c8cbccf403299d3f
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


## Step 2: retrieve data from Github
after executing this part, you should have a variable with dimension (initial season, ens, initial year, forecast time, lat, lon)


*   season = Jan, Apr, Jul, Oct
*   ens    = 15 ensemble members
*   initial year  = 1995-2017
*   forecast time = 0-11 months
*   lat           = 20N-80N
*   lon           = 100E-60N


season = Jan, Apr, Jul, Oct
ens    = 15 ensemble members
initial time = 

In [3]:
# clear current folder
import glob, os
for f in glob.glob("*.npz"):
    os.remove(f)

# define the variables
# clear existing variable in case of duplication
if 'pcs' in locals():
    del pcs
if 'EOF' in locals():
    del EOF
if 'AR_anomaly' in locals():
    del AR_anomaly

pcs        = np.zeros((1000,16560))
EOF        = np.zeros((1000,139,319))

# downloading data from Github
for i in range(1,5):
    url='https://github.com/kuiper2000/kuiper2000.github.io/raw/master/publication/AR_seasonal_prediction/model_EOF'+str(i)+'.npz'
    print(url)
    filename         = wget.download(url)
    data             = np.load('/content/model_EOF'+str(i)+'.npz')
    EOF[250*(i-1):250*(i-1)+250] = data['EOF']
EOF = np.reshape(EOF,[1000,139*319])

for i in range(1,3):
    url='https://github.com/kuiper2000/kuiper2000.github.io/raw/master/publication/AR_seasonal_prediction/model_pcs'+str(i)+'.npz'
    print(url)
    filename         = wget.download(url)
    data             = np.load('/content/model_pcs'+str(i)+'.npz')
    pcs[500*(i-1):500*(i-1)+500] = data['pcs_ar']

AR_anomaly = np.matmul(np.transpose(pcs),EOF)
AR_anomaly = np.reshape(AR_anomaly,[4,15,23,12,139,319])

https://github.com/kuiper2000/kuiper2000.github.io/raw/master/publication/AR_seasonal_prediction/model_EOF1.npz
https://github.com/kuiper2000/kuiper2000.github.io/raw/master/publication/AR_seasonal_prediction/model_EOF2.npz
https://github.com/kuiper2000/kuiper2000.github.io/raw/master/publication/AR_seasonal_prediction/model_EOF3.npz
https://github.com/kuiper2000/kuiper2000.github.io/raw/master/publication/AR_seasonal_prediction/model_EOF4.npz
https://github.com/kuiper2000/kuiper2000.github.io/raw/master/publication/AR_seasonal_prediction/model_pcs1.npz
https://github.com/kuiper2000/kuiper2000.github.io/raw/master/publication/AR_seasonal_prediction/model_pcs2.npz
